In [18]:
from db import db

def print_log():
    for one in db.repka_logs.find():
        print("- ", one["request"])
        print("- ", one["response"])
        print("---")

print_log()


-  кошка
-  Я кошка. Помогу вам. Кошка 🐱 за жучку, жучка 🐶 за внучку, внуч
---
-  крокодила
-  Я крокодил. Помогу вытянуть репку. Крокодил 🐊 за ворону, ворона
---
-  собаку
-  Я собака. Помогу вам. Собака 🐶 за внучку, внучка 👧 за бабку, б
---
-  сову
-  Я сова. Помогу вам. Сова 🦉 за крокодила, крокодил 🐊 за ворону,
---
-  лисичку
-  Я лисичка. Буду помогать. Лисичка 🦊 за сову, сова 🦉 за крокоди
---
-  алиса
-  Я алиса. Помогу вытянуть репку. Алиса 📱 за чарли бакету, чарли 
---
-  эльзу
-  Я эльза. Буду помогать. Эльза 👩‍🦳 за внучку, внучка 👧 за лоша
---
-  мышку
-  Я мышка. Помогу вытянуть репку. Мышка 🐭 за кошку, кошка 🐱 за ж
---
-  
-  Хорошо. Давай поиграем. Ты любишь придумывать сказки? Я – очень 
---
-  
-  Хорошо. Давай поиграем. Ты любишь придумывать сказки? Я – очень 
---
-  бурый мишка
-  Я бурый мишка. Помогу вытянуть репку. Бурый мишка 🐻 за лисичку,
---
-  роботы
-  Я робот. Буду помогать. Робот 🤖 за гарри поттера, гарри поттер 
---
-  кошки кошку позвал
-  Я кошка. Помогу в

In [11]:
db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

{'ns': 'skills.repka_logs',
 'size': 0,
 'count': 3,
 'avgObjSize': 246,
 'storageSize': 0,
 'freeStorageSize': 0,
 'capped': False,
 'wiredTiger': {'metadata': {'formatVersion': 1},
  'creationString': 'access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=snappy,cache_resident=false,checksum=on,colgroups=,collator=,columns=,dictionary=0,encryption=(keyid=,name=),exclusive=false,extractor=,format=btree,huffman_key=,huffman_value=,ignore_in_memory_cache_size=false,immutable=false,import=(enabled=false,file_metadata=,repair=false),internal_item_max=0,internal_key_max=0,internal_key_truncate=true,internal_page_max=4KB,key_format=q,key_gap=10,leaf_item_max=0,leaf_key_max=0,leaf_page_max=32KB,leaf_value_max=64MB,log=(enabled=false),lsm=(auto_throttle=true,bloom=true,bloom_bit_count=16,bloom_config=,bloom_hash_count=8,bloom_oldest=false,c

In [15]:
db.drop_collection('logs')

{'nIndexesWas': 1,
 'ns': 'skills.logs',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1643269647, 1),
  'signature': {'hash': b'\x8e\x16\xf0\xbb\xa4a\xab#ti\xbe\x12\xc6\xbe\xb2\xa4\x90\xfc\xd0I',
   'keyId': 6993093161828483077}},
 'operationTime': Timestamp(1643269647, 1)}